# 18.S096 Pset 2 Solutions, January 2017

The following are sample solutions to pset 2.  Of course, there are
other ways to implement these functions.  The following solutions should be reasonably well optimized, but of course we hope that you come up with even better solutions.

The solutions code is marked with `# SOLUTION` comments.  This way, if you just want to include the solutions in another notebook, you can do:
```jl
using NBInclude
nbinclude("pset2-solutions.ipynb", regex=r"^#\s*SOLUTION")
```
via the [NBInclude](https://github.com/stevengj/NBInclude.jl) package.

In [1]:
using BenchmarkTools, Base.Test

# provide a convenient macro; this will be included in
# BenchmarkTools as @belapsed in the next release
# (https://github.com/JuliaCI/BenchmarkTools.jl/pull/37)
"""
Like `@benchmark`, but returns only the minimum time in ns.
"""
macro benchtime(args...)
    b = Expr(:macrocall, Symbol("@benchmark"), map(esc, args)...)
    :(time(minimum($b)))
end

@benchtime

# Problem 1: Continued Fractions

Assuming the user provided the coefficents as numeric literals, it seems like it should be much better to simplify the continued fraction in to a ratio of two polynomials (a [rational function](https://en.wikipedia.org/wiki/Rational_function)), with precomputed coefficients.  That way, we will only have a single division operation (which is especially expensive for complex numbers), and we can use `@evalpoly` to evaluate the numerator and denominator polynomials.

For non-literal coefficients, we could continue to just call our generic `cf` function as in the pset, but we'll at least unroll the loop to gain a tiny amount of speed:

In [2]:
# function form provided with the pset, for comparison:
function cf(x, a...)
    z = inv(one(x)) # initialize z this way for type stability
    for i = length(a):-1:1
        z = x + a[i] / z
    end
    return inv(z)
end

cf (generic function with 1 method)

In [3]:
# SOLUTION code used for non-literal coefficients:
# we just unroll and inline the loop from above
macro cf(x, a...)
    if isempty(a)
        return :(one(real($(esc(x)))))
    elseif length(a) == 1
        return :(inv($(esc(x)) + $(a[1])))
    end
    # build up expression in terms of let t=x
    # (in case x involves some computation we don't want to repeat)
    # (we use esc, below, in case any)
    z = :(t + $(esc(a[end])))
    for i = length(a)-1:-1:1
        z = :(t + $(esc(a[i])) / $z)
    end
    quote
        let t = $(esc(x))
            inv($z)
        end
    end
end

@cf (macro with 1 method)

In [4]:
@test @cf(3, pi, e) ≈ cf(3, pi, e)

Test Passed
  Expression: @cf(3,pi,e) ≈ cf(3,pi,e)
   Evaluated: 0.2817381941258241 isapprox 0.2817381941258241

We'll do a quick benchmark, putting it in a `sum` loop to make sure that the compiler doesn't just do all the calculations at compile-time:

In [5]:
const α = 2.7
cftest1_fast(x) = @cf x 2 α 4 5 6 7 8
cftest1_slow(x) = cf(x, 2,α,4,5,6,7,8)
x = rand(10^3)
@benchtime(sum(cftest1_slow,$x)) / @benchtime(sum(cftest1_fast,$x))

23.818947525342875

Wow, a factor of 20!

Now, onto the case we will try to optimize further: the one where all the coefficients are constants known at parse time, i.e. numeric literals.

How should we compute the numerator and denominator polynomials?  It seems like a lot of tedious algebra.  Fortunately, we have the SymPy package, which is *very good* at tedious algebra.  Maybe we can get it to do all of the work for us?  (Since the macro expansion occurs when the code is loaded, not when it is run, and can be precompiled in a module, the cost of the SymPy computation is irrelevant.)

In [6]:
using SymPy

y = cf(Sym(:x),3,4,5,6)

        1        
-----------------
          3      
x + -------------
            4    
    x + ---------
              5  
        x + -----
            x + 6

In [7]:
# the macro should give the same expression
@cf(Sym(:x),3,4,5,6)

        1        
-----------------
          3      
x + -------------
            4    
    x + ---------
              5  
        x + -----
            x + 6

Some digging around through the [sympy manual on simplification](http://docs.sympy.org/dev/tutorial/simplification.html) turns up the `cancel` function, which does exactly what we want, along with `numer` and `denom` functions to get the numerator and denominator, and an `all_coeffs` function to get the coefficients:

In [8]:
r = cancel(y)

      3      2               
     x  + 6*x  + 9*x + 24    
-----------------------------
 4      3       2            
x  + 6*x  + 12*x  + 42*x + 15

In [9]:
numer(r)

 3      2           
x  + 6*x  + 9*x + 24

In [10]:
denom(r)

 4      3       2            
x  + 6*x  + 12*x  + 42*x + 15

In [11]:
coefs = all_coeffs(Poly(numer(r))) # need to convert to the Poly class first

4-element Array{Any,1}:
  1
  6
  9
 24

In [12]:
# whoops, gives a symbolic expression, not a number:
typeof(coefs[1])

SymPy.Sym

In [13]:
typeof(N(coefs[1])) # we can use sympy's N(...) to get a number

Int64

In [14]:
N.(coefs)

4-element Array{Int64,1}:
  1
  6
  9
 24

In [15]:
# SOLUTION:
using SymPy
macro cf(x, a::Number...)
    if isempty(a)
        return :(one(real($(esc(x)))))
    elseif length(a) == 1
        return :(inv($(esc(x)) + $(a[1])))
    end
    r = cancel(cf(Sym(:x), a...))
    evalpoly = Symbol("@evalpoly")
    num_expr = Expr(:macrocall, evalpoly, :t, 
                    reverse(N.(all_coeffs(Poly(numer(r)))))...)
    den_expr = Expr(:macrocall, evalpoly, :t, 
                    reverse(N.(all_coeffs(Poly(denom(r)))))...)
    quote
        let t = $(esc(x))
            $num_expr / $den_expr
        end
    end
end

@cf (macro with 2 methods)

In [16]:
@test @cf(1,2,3,4,5) ≈ cf(1, 2,3,4,5)
@test @cf(3) == 1
@test @cf(3,4) == 1/(3+4)

Test Passed
  Expression: @cf(3,4) == 1 / (3 + 4)
   Evaluated: 0.14285714285714285 == 0.14285714285714285

Good, seems to work!  How fast is it?

In [17]:
cftest_fast(x) = @cf x 2 3 4 5 6 7 8
cftest_slow(x) = cf(x, 2,3,4,5,6,7,8)
x = rand(10^3)
@benchtime(sum(cftest_slow, $x)) / @benchtime(sum(cftest_fast, $x))

43.489668845450886

Cool, a 40x speedup, even better than what we got by unrolling above!  For complex numbers, the speedup should be even bigger since complex divisions are so expensive:

In [18]:
z = rand(Complex{Float64},10^3)
@benchtime(sum(cftest_slow, $z)) / @benchtime(sum(cftest_fast, $z))

5.7465942355325375

Hmm, surprisingly not.  What about the non-literal version?

In [19]:
@benchtime(sum(cftest1_slow, $z)) / @benchtime(sum(cftest1_fast, $z))

2.89918941711912

Okay, that's consistent, it doesn't get as much speedup.

# Problem 2: implementing broadcast

I was hoping to define this as `mybroadcast{N}(f::Function, args::Vararg{Union{Number,AbstractVector},N})`, so that the number `N` of arguments is known to the compiler, which would then allow me to use `ntuple` with `Val{N}` as in lecture 3, but I ran into a performance issue with splatting ([#20196](https://github.com/JuliaLang/julia/issues/20196)).

Instead, we'll implement it like the built-in `broadcast` function (in [broadcast.jl](https://github.com/JuliaLang/julia/blob/release-0.5/base/broadcast.jl)), and use [generated functions](http://docs.julialang.org/en/stable/manual/metaprogramming/#generated-functions).  Like macros, these allow you to generate arbitrary code expressions that are inserted into the function to be compiled.  Unlike macros, generated functions are invoked at *compile-time* when the *types* of the arguments are known, and take the *types* as parameters.  In particular.

In [20]:
# SOLUTION:

# special cases that we don't handle in our generated function below
# (omitting these would violate the assert calls below)
mybroadcast(f::Function) = f()
mybroadcast(f::Function, args::Number...) = f(args...)

# rather than lots of "if args[i] <: AbstractVector" checks below,
# it is more natural in Julia just to define methods that do this
# via dispatch.
myeltype{T<:AbstractVector}(a::Type{T}) = eltype(T)
myeltype{T}(a::Type{T}) = T
mylength(a::AbstractVector) = length(a)
mylength(a) = -1
Base.@propagate_inbounds mygetindex(a::AbstractVector, i) = a[i]
@inline mygetindex(a, i) = a

@generated function mybroadcast{F<:Function}(f::F, args::Union{Number,AbstractVector}...)
    assert(!isempty(args)) # empty case should be handled above
    T = promote_type(map(myeltype, args)...)
    Ts = map(myeltype, args)
    
    # create the function-call expression for loop iteration i
    fcall = Expr(:call, :f, ntuple(k -> :(mygetindex(args[$k], i)), length(args))...)
    
    quote
        # compute the length of the result
        # (at runtime, since length of arrays is not part of the type)
        n = -1
        for k in 1:length(args)
            na = mylength(args[k])
            if n == -1
                n = na
            elseif n != na != -1
                throw(DimensionMismatch())
            end
        end
        assert(n >= 0) # should have been at least one vector arg
    
        result = Array{$T}(n) # $T substitutes in the T value computed above
        @inbounds @simd for i = 1:n
            result[i] = $fcall # sub in the fcal expression computed above
        end

        return result
    end
end

mybroadcast (generic function with 3 methods)

In [21]:
mybroadcast(+, 1, 1:10)

10-element Array{Int64,1}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11

In [22]:
mybroadcast(+, 1, 1:3, [10,100,1000])

3-element Array{Int64,1}:
   12
  103
 1004

Hooray, it seems to work!

Now, let's check type inference with `@code_warntype`:

In [23]:
@code_warntype mybroadcast(+, 1, 1:10)

Variables:
  #self#::#mybroadcast
  f::Base.#+
  args::Tuple{Int64,UnitRange{Int64}}
  n::Int64
  result::Array{Int64,1}
  r#322::UnitRange{Int64}
  #temp#@_7::Int64
  i#323::Int64
  n#324::Int64
  i@_10::Int64
  i#325::Int64
  i@_12::Int64
  #temp#@_13::Int64
  k::Int64
  na::Int64
  #temp#@_16::Bool
  #temp#@_17::LambdaInfo
  #temp#@_18::Int64
  #temp#@_19::Void
  ret@_20::Int64
  ret@_21::Int64
  ret@_22::Int64

Body:
  begin  # line 19:
      NewvarNode(:(result::Array{Int64,1}))
      # meta: location In[20] # line 29:
      n::Int64 = -1 # line 30:
      SSAValue(6) = (Base.nfields)(args::Tuple{Int64,UnitRange{Int64}})::Int64
      SSAValue(12) = (Base.select_value)((Base.sle_int)(1,SSAValue(6))::Bool,SSAValue(6),(Base.box)(Int64,(Base.sub_int)(1,1)))::Int64
      #temp#@_13::Int64 = 1
      10: 
      unless (Base.box)(Base.Bool,(Base.not_int)((#temp#@_13::Int64 === (Base.box)(Int64,(Base.add_int)(SSAValue(12),1)))::Bool)) goto 51
      SSAValue(13) = #temp#@_13::Int64
      SSA

Good, all of the variables, and also the return value, are marked as having concrete types.

Now, let's time it against the built-in `broadcast` function:

In [24]:
x = rand(10000)
@test broadcast(+, x, 1) == mybroadcast(+, x, 1)
@benchtime(mybroadcast(+, $x, 1)) / @benchtime(broadcast(+, $x, 1))

0.904869576615546

Hooray, it is the same speed, and in fact slightly faster (probably due to the fact that it is much less general)!

Since the `broadcast` function is highly optimized, this is a good indication that we have done about as well as we can do with a reasonable amount of effort, so let's declare victory and move on to the next problem.

# Problem 3: Linear algebra

The first key thing is to realize that

$$
A = \begin{pmatrix} 
        2 & -1 &  &  &  \\
        -1 & 2 & -1 &  &  \\
         & \ddots & \ddots & \ddots &   \\
         &  & -1 & 2 & -1  \\
         &  &  & -1 & 2 
    \end{pmatrix} + \begin{pmatrix} 
        1 & 1 & 1 & 1 & \cdots \\
        1 & 1 & 1 & 1 & \cdots \\
        1 & 1 & 1 & 1 & \cdots \\
        \vdots & \vdots & \vdots & \vdots & \cdots \\
        1 & 1 & 1 & 1 & 1
    \end{pmatrix}
= T + oo^T
$$

where $T$ is a real-symmetric positive-definite matrix (discussed in lecture) and $o$ is a vector of 1's.   This is a **rank-1 update** to $T$.

We can multply $Ax = Tx + o(o^T x)$ in $O(m)$ operations, since $T$ is sparse and $o^T x$ is just a dot product, so one approach might be to use some kind of iterative method like preconditioned conjugate-gradient (since $A$ is real-symmetric positive-definite).

However, since we can multiply by $T^{-1}$ quickly, in $O(m)$ time in fact, we can use the [Sherman–Morrison formula](https://en.wikipedia.org/wiki/Sherman%E2%80%93Morrison_formula) to apply the inverse of the rank-1 update $A$:

$$
(T + oo^T)^{-1}b = T^{-1}b - \frac{T^{-1}o o^T T^{-1}b}{1 + o^T T^{-1} o}
$$
which requires us to solve only $T^{-1} b$ and $T^{-1} o$ and then do some dot products and other simple operations.

Let's implement this.

In [25]:
# The slow O(m^3) method included with the pset, for comparison.

function mymatrix{T}(::Type{T}, m::Int)
    A = ones(T, m, m)
    for i = 1:m
        A[i,i] = 3
    end
    for i = 1:m-1
        A[i,i+1] = A[i+1,i] = 0
    end
    return A
end
      
mymatrix(m::Integer) = mymatrix(Int, Int(m))

# solve Ax = b, returning x, for the "mostly ones" matrix A above
function mysolve_slow{T<:Number}(b::AbstractVector{T})
    m = length(b)
    A = mymatrix(float(T), m)
    return A \ b
end

mysolve_slow (generic function with 1 method)

Here's our first try at an O(m) method, with the most obvious implementation, exploiting Julia's `SymTridiagonal` matrix type for $T$:

In [26]:
function mysolve1{S<:Number}(b::AbstractVector{S})
    m = length(b)
    T = SymTridiagonal([2 for i=1:m], [-1 for i=1:m-1])
    o = ones(b)
    T⁻¹o = T \ o
    T⁻¹b = T \ b
    return T⁻¹b - T⁻¹o * (dot(o,T⁻¹b) / (1 + dot(o,T⁻¹o)))
end

mysolve1 (generic function with 1 method)

In [27]:
b = rand(1000)
@test mysolve_slow(b) ≈ mysolve1(b)
bench_slow = @benchmark mysolve_slow($b)

BenchmarkTools.Trial: 
  memory estimate:  15.27 MiB
  allocs estimate:  12
  --------------
  minimum time:     13.857 ms (0.00% GC)
  median time:      15.393 ms (9.80% GC)
  mean time:        15.136 ms (8.19% GC)
  maximum time:     17.822 ms (8.43% GC)
  --------------
  samples:          330
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [28]:
bench1 = @benchmark mysolve1($b)

BenchmarkTools.Trial: 
  memory estimate:  87.80 KiB
  allocs estimate:  33
  --------------
  minimum time:     45.918 μs (0.00% GC)
  median time:      53.191 μs (0.00% GC)
  mean time:        67.808 μs (11.70% GC)
  maximum time:     3.913 ms (97.44% GC)
  --------------
  samples:          10000
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

This is almost a factor of 1000 improvement in both memory and time, which is not surprising.  If anything, it is amazing that the improvement is not greater, considering that we went from $O(m^3)$ to $O(m)$ complexity.

There are various ways to make it faster:

* `T \ o` and `T \ b` actually [computes a factorization](https://github.com/JuliaLang/julia/blob/079b1c3b18c683c9aa1282aa159f462177074312/base/linalg/tridiag.jl#L163) (an $L D L^T$ factorization) of `T` twice.  (Moreover, it doesn't take advantage of the fact that `T` is positive-definite.)

* `dot(o,x)` could be replaced with simply `sum(x)`.

* Our final expression allocates a bunch of temporary arrays.  We should fuse the loops and allocate only a single output array.  (This will be easier in Julia 0.6 due to the [new fused vectorization syntax](http://julialang.org/blog/2017/01/moredots).)

* Since the entries of `T` are just 2 and -1, we shouldn't actually need to store `T` at all.  We should be able to hard-code the factorization computation specifically for this matrix if we are ambitious.  It's helpful here to look at the [source code for the factorization of `SymTridiagonal`](https://github.com/JuliaLang/julia/blob/079b1c3b18c683c9aa1282aa159f462177074312/base/linalg/ldlt.jl#L18-L77).

Let's addres the first three issues, since they are easy:

In [29]:
# SOLUTION:

function mysolve{S<:Number}(b::AbstractVector{S})
    m = length(b)
    Sf = typeof(inv(one(S))) # compute type of the output arrays
                             # (this will be inferred at compile-time)
    T = SymTridiagonal(Sf[2 for i=1:m], Sf[-1 for i=1:m-1])
    LDLᵀ = ldltfact!(T) # compute factorization in-place, overwriting T
    T⁻¹o = A_ldiv_B!(LDLᵀ, ones(Sf, m)) # overwrites ones array in-place
    T⁻¹b = LDLᵀ \ b
    α = sum(T⁻¹b) / (1 + sum(T⁻¹o))
    x = Vector{Sf}(m)
    @inbounds @simd for i = 1:m
        x[i] = T⁻¹b[i] - T⁻¹o[i] * α
    end
    return x
end

mysolve (generic function with 1 method)

In [30]:
@test mysolve(b) ≈ mysolve1(b)
bench2 = @benchmark mysolve($b)
judge(minimum(bench2), minimum(bench1)) # compare the two results

BenchmarkTools.TrialJudgement: 
  time:   -47.28% => improvement (5.00% tolerance)
  memory: -54.60% => improvement (1.00% tolerance)

Almost a factor of two improvement in both time and memory allocation, which is nice but not overwhelming.  It's probably not worth the trouble to try to squeeze any more speed out of this function, though marginal improvements are no doubt possible.

(A good check, not shown here, is to run `@code_warntype mysolve(b)` to make sure that the compiler is able to infer all of the types in the function.  You should find that the result type is correctly inferred as `Vector{Float64}`, and there are no other type instabilities.)